В данном ноутбуке представлено моделирование бейзлайна на основе общих данных, подготовленных в ноутбуке [2.2-mv-eda-feature-selection.ipynb](https://github.com/mvulf/housing_cost/blob/main/notebooks/2.2-mv-eda-feature-selection.ipynb). Используется линейная регрессия и дерево решений.

Используются MLFlow и Pipeline, как описано в предыдущем ноутбуке (3.0-mv-modelling-init.ipynb)

Оценка результатов моделирования проводится как на 5-ти фолдовой кросс-валидации, так и на финальном тесте.
Кросс-валидация проводится на десятично логарифме цены, так как именно он и предсказывается. 
Метрики по финальному тесту строятся уже на действительной цене, полученной возведением 10 в предсказанную степень.  

Начиная с этого ноутбука и далее, модели участвуют в финальной сравнительной таблице в README.md.

Подготовим бейзлайн для сравнения остальных моделей с ним

# Импорт библиотек и данных

Launch a server via:
```bash
mlflow server --host 127.0.0.1 --port 8080
```

In [1]:
import sys
from pathlib import Path

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import mlflow
from mlflow.models import infer_signature

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_validate,\
    GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

root_folder = '../'
sys.path.append(root_folder)

from src.visualization import visualize
from src.models import train_model, predict_model
from src.utils import get_dict

train_path = Path(root_folder, 'data', 'processed', '2.0_train.csv')
test_path = Path(root_folder, 'data', 'processed', '2.0_test.csv')

experiment_name = 'Housing cost'

Загрузим датасеты:

In [2]:
train = pd.read_csv(train_path, index_col=0)
train.info()
X_train, y_train = train_model.get_X_y(train, target_name='log_target')
print()
print()

test = pd.read_csv(test_path, index_col=0)
test.info()
X_test, y_test = train_model.get_X_y(test, target_name='log_target')

<class 'pandas.core.frame.DataFrame'>
Index: 264639 entries, 0 to 264638
Data columns (total 41 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   baths                            264639 non-null  float64
 1   fireplace                        264639 non-null  bool   
 2   beds                             264639 non-null  float64
 3   stories                          264639 non-null  float64
 4   private_pool                     264639 non-null  bool   
 5   parking_count                    264639 non-null  float64
 6   central_heating                  264639 non-null  bool   
 7   central_cooling                  264639 non-null  bool   
 8   log_target                       264639 non-null  float64
 9   log_sqft                         264639 non-null  float64
 10  log_lotsize                      264639 non-null  float64
 11  updated_years                    264639 non-null  float64
 12  school_

# Baseline

## Linear regression

Проверим линейную регрессию

In [3]:
# Prepare pipe
imputer_params = get_dict(
    missing_values=np.nan,
    strategy='median'
)
pipe_elements = [
    ('imputer', SimpleImputer, imputer_params),
    ('scaler', MinMaxScaler),
    ('regressor', LinearRegression)
]
pipe, pipe_params = train_model.make_pipeline(pipe_elements)
display(pipe)

# Conduct fitting and cross-validation metrics estimation
cv_metrics = predict_model.cross_validate_pipe(
    pipe=pipe,
    X=X_train,
    y=y_train,
)

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('scaler', MinMaxScaler()), ('regressor', LinearRegression())])

,cv_train,cv_validation
mape_log,0.036,0.036
r2_log,0.505,0.505


Обучим и протестируем модель

In [4]:
pipe.fit(X_train, y_train)
metrics = predict_model.get_train_test_metrics(
    pipe,
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
)
metrics = metrics | cv_metrics

,train,test
mape,0.530,0.531
r2,0.393,0.391


Проверим, что качественно ошибка в логарифме так соотносится с ошибкой в абсолютной цене, как метрики логарифма и реальной цены:

In [5]:
true_median = y_train.median()
mape_log = 0.036
mape = 10 ** (mape_log*true_median) - 1
mape

0.5800342525040665

Да, порядок верный. Разница возникает в связи с тем, что в действительности осреднение идёт по всей сумме ошибок. А оценка проведена по медианному значению

In [6]:
model_info = predict_model.log_pipe_mlflow(
    pipe_name='baseline-linreg',
    training_info='Linear regression baseline after eda',
    X=X_train,
    pipe=pipe,
    pipe_params=pipe_params,
    metrics=metrics,
    experiment_name=experiment_name,
)

2024/04/24 17:51:34 INFO mlflow.tracking.fluent: Experiment with name 'Housing cost' does not exist. Creating a new experiment.


Registered model 'baseline-linreg' already exists. Creating a new version of this model...
2024/04/24 17:51:45 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: baseline-linreg, version 2
Created version '2' of model 'baseline-linreg'.


## Dtree

Методом GridSearchCV подберём лучшую глубину дерева

In [7]:
# Prepare pipe
imputer_params = get_dict(
    missing_values=np.nan,
    strategy='median'
)
dtree_params = get_dict(
    # max_depth=20,
    random_state=42
)
pipe_elements = [
    ('imputer', SimpleImputer, imputer_params),
    ('scaler', MinMaxScaler),
    ('regressor', DecisionTreeRegressor, dtree_params)
]
pipe, pipe_params = train_model.make_pipeline(pipe_elements)
display(pipe)

max_depth_grid = [2, 4, 8, 10, 12, 16, 20, 24, 28, 32, 64, 128]
grid = GridSearchCV(
    pipe,
    param_grid={
        'regressor__max_depth': max_depth_grid,
    },
    scoring=(
        'neg_mean_absolute_error',
        'r2'
    ),
    refit='r2',
    n_jobs=-1,
    return_train_score=True,
)
grid.fit(X_train, y_train)
# print(f'Best mean absolute error: {-grid.best_score_:.3e}')
print(f'Best r2-score: {grid.best_score_:.3f}')
print(f'Best params:', grid.best_params_)

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('scaler', MinMaxScaler()),
                ('regressor', DecisionTreeRegressor(random_state=42))])

Best r2-score: 0.565
Best params: {'regressor__max_depth': 12}


Проведём кросс-валидацию на модели с оптимизированной глубиной дерева

In [8]:
# random_state = 42

# Prepare pipe
imputer_params = get_dict(
    missing_values=np.nan,
    strategy='median'
)
dtree_params = get_dict(
    max_depth=grid.best_params_['regressor__max_depth'],
    random_state=42
)
pipe_elements = [
    ('imputer', SimpleImputer, imputer_params),
    ('scaler', MinMaxScaler),
    ('regressor', DecisionTreeRegressor, dtree_params)
]
pipe, pipe_params = train_model.make_pipeline(pipe_elements)
display(pipe)

# Conduct fitting and cross-validation metrics estimation
cv_metrics = predict_model.cross_validate_pipe(
    pipe=pipe,
    X=X_train,
    y=y_train,
)

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('scaler', MinMaxScaler()),
                ('regressor',
                 DecisionTreeRegressor(max_depth=12, random_state=42))])

,cv_train,cv_validation
mape_log,0.031,0.033
r2_log,0.621,0.565


Обучим и протестируем модель

In [9]:
pipe.fit(X_train, y_train)
metrics = predict_model.get_train_test_metrics(
    pipe,
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
)
metrics = metrics | cv_metrics

,train,test
mape,0.448,0.492
r2,0.586,0.492


Логируем дерево решений

In [10]:
model_info = predict_model.log_pipe_mlflow(
    pipe_name='baseline-dtree',
    training_info='Decision tree baseline after eda',
    X=X_train,
    pipe=pipe,
    pipe_params=pipe_params,
    metrics=metrics,
    experiment_name=experiment_name,
)

/home/vulf/.cache/pypoetry/virtualenvs/housing-cost-EeioB1ZZ-py3.11/lib/python3.11/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/vulf/.cache/pypoetry/virtualenvs/housing-cost-EeioB1ZZ-py3.11/lib/python3.11/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'baseline-dtree' already exists. Creating a new version of this model...
2024/04/24 17:53:11 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation.

## Выводы по бейзлайнам c EDA

После проведения разведывательного анализа данных удалось улучшить показатели метрик.

Фаворитом по-прежнему остаётся дерево решений

Показатели лучшей модели (дерева решений) на финальном тесте:
- **MAPE**: $0.49$ (то есть ошибка $49\%$)
- **R^2**: $0.49$